In [ ]:
from rdflib import Graph
import pandas as pd
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
HUGGINGFACE_APIKEY = os.getenv('HUGGINGFACE_APIKEY')

In [ ]:
g = Graph()
g.parse("../pipeline/3_mergeOutput/UnifiedOntology.ttl")

In [ ]:
query = f"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?classLabel 
WHERE {{
    ?class a rdfs:Class .
    ?class rdfs:label ?classLabel .
}}

"""

In [ ]:
qres = g.query(query)
df = pd.DataFrame(qres.bindings)
df.columns = df.columns.str.replace(' ', '')
df[:3]

In [ ]:
from transformers import AutoModelForSequenceClassification, BertTokenizer, pipeline

In [ ]:
!huggingface-cli login --token {HUGGINGFACE_APIKEY}

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("OhWayTee/bert-taxonomy")

In [ ]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
predictions = []
classA = ''
classB = ''
for idx, row in df.iterrows():
    classA = row.iloc[0]
    for idx, row in df.iterrows():
        classB = row.iloc[0]
        prediction = classifier(f"{classA} [SEP] {classB}")
        prediction[0].update({'classA': classA, 'classB' : classB})
        predictions.append(prediction[0])   

In [ ]:
df_pred = pd.DataFrame(predictions) 
df_pred[:5]

In [ ]:
df_pred = df_pred[df_pred.label != "LABEL_0"].sort_values(by=['score'], ascending=False)
df_pred